In [109]:
import pandas as pd
import numpy as np

In [110]:
ground = pd.read_csv('BRITS/air_raw/pm25_ground.txt')
missing = pd.read_csv('BRITS/air_raw/pm25_missing.txt')
ground1 = ground.drop('datetime',axis=1)

In [115]:
out_array = ground1.to_numpy()

In [151]:
validation_ranges = list(range(744,1463)) + list(range(2951,3671)) + list(range(5135,5879)) + list(range(7295,8039))
train_ranges = list(set(range(ground.shape[0]))-set(validation_ranges))
train_ranges.sort()
train_ranges = train_ranges[50:-50]

In [103]:
final_validation_points = []
sensors = ['001001', '001002', '001003', '001004', '001005', '001006',
       '001007', '001008', '001009', '001010', '001011', '001012',
       '001013', '001014', '001015', '001016', '001017', '001018',
       '001019', '001020', '001021', '001022', '001023', '001024',
       '001025', '001026', '001027', '001028', '001029', '001030',
       '001031', '001032', '001033', '001034', '001035', '001036']

def prev_month(date='2015/09/01 00:00:00'):
    prev_month = int(date[5:7])-1
    
    needed_str = '0%d'%prev_month if prev_month<10 else '%d'%prev_month
    return date[:5]+needed_str+date[7:]

for x in validation_ranges:
    try :
        t = ground.iloc[x]
        prev = ground.loc[ground['datetime'] == prev_month(t['datetime'])].iloc[0]
        for y in sensors :
            if (not np.isnan(t[y]) and (not np.isnan(prev[y]))):
                final_validation_points.append((x,y))
    except Exception as e:
        continue
final_validation_points = [[x[0],int(x[1])-1001] for x in final_validation_points]

In [152]:
final_train_points = []
for i in train_ranges:
    for j in range(36):
        if (not np.isnan(out_array[i][j])):
            final_train_points.append([i,j])

In [153]:
all_examples = []
for i in range(50,out_array.shape[0]-50):
    for j in range(out_array.shape[1]):
        all_examples.append([i,j])

In [154]:
np.save('dataset/1dair_complete',out_array)
np.save('dataset/1dair_test_examples.npy',np.array(final_validation_points))
np.save('dataset/1dair_train_examples.npy',np.array(final_train_points))
np.save('dataset/1dair_all_examples.npy',np.array(all_examples))